In [1]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

sys.path.append('../')
#from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *
from get_address_points import *

In [2]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [3]:
geo_key = config['keys']['geosupport_key']
geo_ip = config['keys']['geosupport_ip']

In [4]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                 ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [5]:
sql = 'select top 5 bin from structuresdb.dbo.tbl_parks_structures'

In [6]:
b = pd.read_sql(con=engine, sql=sql)

In [7]:
bins = list(b[b['bin'].notnull()]['bin'])

In [8]:
aps = get_address_point(bins, geom_col = 'the_geom')[0]

In [9]:
aps

,the_geom,bin,h_no,address_id,hyphen_typ,side_of_st,borocode,zipcode,created,st_name,physicalid,post_type,full_stree,modified
0,POINT (995998.149 204776.450),3400033,81,5168623,N,1,3,11222,2015-12-15T00:00:00.000Z,FRANKLIN,64739,ST,FRANKLIN ST,NaN
1,POINT (984261.245 157105.052),3170079,2134,3217153,N,2,3,11214,2009-02-13T00:00:00.000Z,CROPSEY,42457,AVE,CROPSEY AVE,NaN
2,POINT (983853.114 156079.787),3340691,9000,3217154,N,2,3,11214,2009-02-13T00:00:00.000Z,BAY,51350,PKWY,BAY PKWY,2018-07-26T00:00:00.000Z


In [10]:
grc_err = ['01/F', '20', '21', '22', '23']
out_keys = [
    'AddressRangeList', 'out_bbl', 'out_TPAD_bin', 'out_TPAD_bin_status',
    'out_TPAD_conflict_flag', 'out_error_message', 'out_grc',
    'out_sanborn_boro', 'in_bin', 'out_boro_name1']

In [11]:
bn = funcbn(bn = bins[2], out_keys = out_keys, grc_err = grc_err, api_key = geo_key, ip = geo_ip)

https://geoservice.planning.nyc.gov/geoservice/geoservice.svc/Function_BIN?BIN=3340691.0&key=UPhZX5HAK72P4MDM
response
raw_dict
{'AddressRangeKeys': [{'Key': '', 'Value': 'Ordinary Address Range'}], 'AddressRangeList': [{'b7sc': '31733001', 'bin': '3340691', 'high_address_number': '            9000', 'low_address_number': '            8922', 'street_name': 'BAY PARKWAY                     ', 'tpad_bin_status': 'N/A', 'type': ' ', 'type_meaning': 'Ordinary Address Range'}], 'in_bin': '3340691', 'in_func_code': 'BN', 'out_TPAD_bin': 'N/A', 'out_TPAD_bin_status': 'N/A', 'out_TPAD_conflict_flag': 'N/A', 'out_bbl': '3064900024', 'out_bbl_block': '6490', 'out_bbl_boro': None, 'out_bbl_lot': '24', 'out_bid': '', 'out_bin': '3340691', 'out_bin_status': 'N/A', 'out_boro_name1': 'BROOKLYN ', 'out_condo_flag': 'Non-Condo', 'out_condo_num': 'N/A', 'out_coop_num': 'N/A', 'out_corner_code': 'CR', 'out_dcp_zoning_map': '28A', 'out_error_message': '                                                     

In [12]:
bn

[{'b7sc': '31733001',
  'bin': '3340691',
  'high_address_number': '            9000',
  'low_address_number': '            8922',
  'street_name': 'BAY PARKWAY                     ',
  'tpad_bin_status': 'N/A',
  'type': ' ',
  'type_meaning': 'Ordinary Address Range',
  'out_bbl': '3064900024',
  'out_TPAD_bin': 'N/A',
  'out_TPAD_bin_status': 'N/A',
  'out_TPAD_conflict_flag': 'N/A',
  'out_error_message': '                                                                                ',
  'out_grc': '00',
  'out_sanborn_boro': '3',
  'in_bin': '3340691',
  'out_boro_name1': 'BROOKLYN '}]

In [13]:
a = aps.iloc[0][['borocode', 'full_stree']]

In [18]:
aps.values

array([[<shapely.geometry.point.Point object at 0x0000000009960188>,
        '3400033', '81', '5168623', 'N', '1', '3', '11222',
        '2015-12-15T00:00:00.000Z', 'FRANKLIN', '64739', 'ST',
        'FRANKLIN ST', nan],
        '3170079', '2134', '3217153', 'N', '2', '3', '11214',
        '2009-02-13T00:00:00.000Z', 'CROPSEY', '42457', 'AVE',
        'CROPSEY AVE', nan],
        '3340691', '9000', '3217154', 'N', '2', '3', '11214',
        '2009-02-13T00:00:00.000Z', 'BAY', '51350', 'PKWY', 'BAY PKWY',
        '2018-07-26T00:00:00.000Z']], dtype=object)

In [14]:
func1n(borough = a['borocode'], streetname = a['full_stree'], api_key = geo_key, ip = geo_ip)

{'out_boro_name1': 'BROOKLYN ',
 'out_grc': '00',
 'out_stname1': 'FRANKLIN STREET                 '}